In [12]:
from __future__ import absolute_import
from __future__ import print_function
from __future__ import unicode_literals

import pexpect
import sys

In [98]:
child = pexpect.spawnu('jdb Test')
child.sendline("stop in Test.main")
child.expect(".*Deferring breakpoint.*")
child.sendline("run")
child.expect(".*Breakpoint hit:")

0

In [103]:
child.before + child.after

'\r\nx = 10\r\nmain[1] '

In [102]:
child.sendline("locals")
child.expect("Method arguments:")
child.expect("Local variables:")
child.expect(".*\[.*\] ")

0

In [101]:
child.sendline("step in")

8

In [66]:
child.expect("Local variables:")

0

In [67]:
child.before

' "thread=main", Test.main(), line=4 bci=0\r\n4    \tint x = 10;\r\n\r\nmain[1] locals\r\nMethod arguments:\r\nargs = instance of java.lang.String[0] (id=495)\r\n'

In [16]:
child.sendline("run")

4

In [142]:
child.close()

In [13]:
def parse_rawstr(rawstr):
    # Remove /r
    rawstr = rawstr.replace('\r\n', '\n')
    
    # Strip trailing spaces
    rawstr = rawstr.strip()
    
    # Split into lines
    lines = rawstr.splitlines()
    
    # Parse each line
    variables = []
    for line in lines:
        if '=' not in line:
            continue
        parts = line.split(sep='=', maxsplit=1)
        name = parts[0].strip()
        value = parts[1].strip()
        variables.append((name, value))
    
    return variables

In [14]:
def step(child):
    if child.closed or child.eof():
        return None
        
    # Make a step
    child.sendline("step in")
    
    # Print out all local variables
    child.sendline("locals")
    
    # Expect the variables from method arguments
    child.expect("Method arguments:")
    
    # Expect the variables locally declared
    child.expect("Local variables:")
    
    rawstr_args = child.before
    
    # Seek forward to prompt
    child.expect(".*\[.*\] ")
    
    rawstr_locals = child.after
    
    # Parse the strings
    args = parse_rawstr(rawstr_args)
    local = parse_rawstr(rawstr_locals)
    
    return (args, local)

In [16]:
try:
    child.close()
except:
    pass

child = pexpect.spawnu('jdb IterPower')
child.sendline("stop in IterPower.main")
child.expect(".*Deferring breakpoint.*")
child.sendline("run")
child.expect(".*Breakpoint hit:")

0

In [22]:
step(child)

([('base', '10'), ('exp', '3')], [('result', '10')])

pexpect.exceptions.EOF

In [9]:
child.after

pexpect.exceptions.EOF

In [199]:
child.readline()

'4    \tint x = 10;\r\n'

In [205]:
child.sendline("")
child.expect(".*\[.*\] ")
child.after

'\r\nmain[1] '

In [207]:
child.after.strip()

'main[1]'

In [230]:
child.buffer

'dump arg3\r\n arg3 = "adding"\r\nmain[1] '

In [291]:
def dump(child, obj):
    
    # First get a prompt
    child.sendline("")
    child.expect(".*\[.*\] ")
    rawstr_prompt =  child.after.strip()
    
    # Escape prompt
    esc_prompt = rawstr_prompt
    esc_prompt = esc_prompt.replace("[", "\[")
    esc_prompt = esc_prompt.replace("]", "\]")
    
    # Send command
    child.sendline("dump {}".format(obj))
    
    # Expect output
    pattern = "{obj} = .*{prompt}".format(
        obj=obj,
        prompt=esc_prompt)
    child.expect(pattern)
    
    # Parse output
    ret = child.after
    
    # - remove last line break and prompt that follows
    last_line_break = ret.rfind("\n")
    ret = ret[:last_line_break].strip()
    
    # - remove obj name and equal sign
    ret = ret.replace("{} =".format(obj), "").strip()
    
    if ret == "null":
        ret = None
    
    return ret

In [293]:
dump(child, "Test.glob")

'2'

In [256]:
child.expect("arg3 = .*main\[1\]")

0

In [259]:
ret = child.after

In [262]:
ret.replace("\r\nmain[1]", "").strip()

'arg3 = "adding"'

In [268]:
"main[1]                main[1]".rfind("main[1]")

23

In [270]:
ret = child.after
last_line_break = ret.rfind("\n")
ret = ret[:last_line_break].strip()

In [274]:
ret.replace("{} =".format("arg3"), "").strip()

'"adding"'

True

In [339]:
import os

In [340]:
os.getcwd()

'/private/tmp'